<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Spark-MLlib-Tuning" data-toc-modified-id="Spark-MLlib-Tuning-1"><span class="toc-item-num">1&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/latest/ml-tuning.html" target="_blank">Spark MLlib Tuning</a></a></span></li><li><span><a href="#Hyperopt" data-toc-modified-id="Hyperopt-2"><span class="toc-item-num">2&nbsp;&nbsp;</span><a href="https://github.com/hyperopt/hyperopt" target="_blank">Hyperopt</a></a></span><ul class="toc-item"><li><span><a href="#XGBoost-Tuning" data-toc-modified-id="XGBoost-Tuning-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span><a href="https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/" target="_blank">XGBoost Tuning</a></a></span><ul class="toc-item"><li><span><a href="#Objective-function" data-toc-modified-id="Objective-function-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Objective function</a></span></li><li><span><a href="#Tune-number-of-trees" data-toc-modified-id="Tune-number-of-trees-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Tune number of trees</a></span></li><li><span><a href="#Tune-tree-specific-parameters" data-toc-modified-id="Tune-tree-specific-parameters-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>Tune tree-specific parameters</a></span><ul class="toc-item"><li><span><a href="#Tune-max_depth,-min_child_weight" data-toc-modified-id="Tune-max_depth,-min_child_weight-2.1.3.1"><span class="toc-item-num">2.1.3.1&nbsp;&nbsp;</span>Tune max_depth, min_child_weight</a></span></li><li><span><a href="#Tune-gamma" data-toc-modified-id="Tune-gamma-2.1.3.2"><span class="toc-item-num">2.1.3.2&nbsp;&nbsp;</span>Tune gamma</a></span></li><li><span><a href="#Tune-subsample,-colsample_bytree" data-toc-modified-id="Tune-subsample,-colsample_bytree-2.1.3.3"><span class="toc-item-num">2.1.3.3&nbsp;&nbsp;</span>Tune subsample, colsample_bytree</a></span></li></ul></li><li><span><a href="#Tune-regularization-parameters" data-toc-modified-id="Tune-regularization-parameters-2.1.4"><span class="toc-item-num">2.1.4&nbsp;&nbsp;</span>Tune regularization parameters</a></span></li><li><span><a href="#Lower-the-learning-rate-and-decide-the-optimal-parameters" data-toc-modified-id="Lower-the-learning-rate-and-decide-the-optimal-parameters-2.1.5"><span class="toc-item-num">2.1.5&nbsp;&nbsp;</span>Lower the learning rate and decide the optimal parameters</a></span></li></ul></li><li><span><a href="#LogisticRegression-Tuning" data-toc-modified-id="LogisticRegression-Tuning-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>LogisticRegression Tuning</a></span></li><li><span><a href="#Optional-MongoTrials" data-toc-modified-id="Optional-MongoTrials-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Optional <a href="https://hyperopt.github.io/hyperopt/scaleout/mongodb/" target="_blank">MongoTrials</a></a></span><ul class="toc-item"><li><span><a href="#XGBoost-Tuning" data-toc-modified-id="XGBoost-Tuning-2.3.1"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span>XGBoost Tuning</a></span></li></ul></li></ul></li><li><span><a href="#Results" data-toc-modified-id="Results-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Results</a></span></li></ul></div>

Продолжаем работать над задачей CTR-prediction с использованием датасета от Criteo.

Описание задачи и данных можно посмотреть в notebook'e предыдущей практики (`sgd_logreg_nn/notebooks/ctr_prediction_mllib.ipynb`).

In [4]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'

import os
import sys
import glob
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import pyspark
import pyspark.sql.functions as F
from pyspark.conf import SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.ml.classification import LogisticRegression

COMMON_PATH = '/workspace/common'

sys.path.append(os.path.join(COMMON_PATH, 'utils'))

os.environ['PYSPARK_SUBMIT_ARGS'] = """
--jars {common}/xgboost4j-spark-0.72.jar,{common}/xgboost4j-0.72.jar
--py-files {common}/sparkxgb.zip pyspark-shell
""".format(common=COMMON_PATH).replace('\n', ' ')

spark = SparkSession \
    .builder \
    .master('local[*]') \
    .appName("spark_sql_examples") \
    .config("spark.executor.memory", "6g") \
    .getOrCreate()

sc = spark.sparkContext
sqlContext = SQLContext(sc)

from metrics import rocauc, logloss, ne, get_ate
from processing import split_by_col

from sparkxgb.xgboost import *

In [5]:
DATA_PATH = '/workspace/data/criteo'

TRAIN_PATH = os.path.join(DATA_PATH, 'train.csv')

In [6]:
df = sqlContext.read.format("com.databricks.spark.csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('file:///' + TRAIN_PATH)

**Remark** Необязательно использовать половину датасета и всего две категориальные переменные. Можно использовать больше данных, если вам позволяет ваша конфигурация

In [7]:
df = df.sample(False, 0.5)

In [8]:
num_columns = ['_c{}'.format(i) for i in range(1, 14)]
cat_columns = ['_c{}'.format(i) for i in range(14, 40)][:2]
len(num_columns), len(cat_columns)

(13, 2)

In [9]:
df = df.fillna(0, subset=num_columns)

In [10]:
from pyspark.ml import PipelineModel


pipeline_model = PipelineModel.load(os.path.join(DATA_PATH, 'pipeline_model'))

In [11]:
df = pipeline_model \
    .transform(df) \
    .select(F.col('_c0').alias('label'), 'features', 'id') \
    .cache()

df.count()

1832893

In [12]:
train_df, val_df, test_df = split_by_col(df, 'id', [0.8, 0.1, 0.1])

# [Spark MLlib Tuning](https://spark.apache.org/docs/latest/ml-tuning.html)

У имеющегося в Spark'e метода HPO есть два существенных недостатка, которые делают его мало пригодным в контексте нашей задачи:

1. `ParamGridBuilder` - поиск по сетке
2. `TrainValidationSplit` - делит данные случайнм образом

# [Hyperopt](https://github.com/hyperopt/hyperopt)

Установим `hyperopt`

In [13]:
!pip3.5 install hyperopt

## [XGBoost Tuning](https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/)

> [Notes on Parameter Tuning](https://xgboost.readthedocs.io/en/latest/tutorials/param_tuning.html)

### Objective function

In [14]:
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
import scipy.stats as st


def objective(space):
    estimator = XGBoostEstimator(**space)
    print('SPACE:', estimator._input_kwargs_processed())
    success = False
    attempts = 0
    model = None
    while not success and attempts < 2:
        try:
            model = estimator.fit(train_df)
            success = True
        except Exception as e:
            attempts += 1
            print(e)
            print('Try again')
        
    log_loss = logloss(model, val_df, probabilities_col='probabilities')
    roc_auc = rocauc(model, val_df, probabilities_col='probabilities')
    
    print('LOG-LOSS: {}, ROC-AUC: {}'.format(log_loss, roc_auc))

    return {'loss': log_loss, 'rocauc': roc_auc, 'status': STATUS_OK }

In [15]:
static_params = {
    'featuresCol': "features", 
    'labelCol': "label", 
    'predictionCol': "prediction",
    'eval_metric': 'logloss',
    'objective': 'binary:logistic',
    'nthread': 1,
    'silent': 0,
    'nworkers': 1
}

Fix baseline parameters and train baseline model

In [16]:
CONTROL_NAME = 'xgb baseline'

baseline_params = {
    'colsample_bytree': 0.9,
    'eta': 0.15,
    'gamma': 0.9,
    'max_depth': 6,
    'min_child_weight': 50.0,
    'subsample': 0.9,
    'num_round': 20
}

baseline_model = XGBoostEstimator(**{**static_params, **baseline_params}).fit(train_df)

In [17]:
baseline_rocauc = rocauc(baseline_model, val_df, probabilities_col='probabilities')
baseline_rocauc

0.726496408208172

In [18]:
all_metrics = {}

In [19]:
baseline_test_metrics = {
    'logloss': logloss(baseline_model, test_df, probabilities_col='probabilities'),
    'rocauc': rocauc(baseline_model, test_df, probabilities_col='probabilities')
}

all_metrics[CONTROL_NAME] = baseline_test_metrics

### Tune number of trees

> Choose a relatively high learning rate. Generally a learning rate of 0.1 works but somewhere between 0.05 to 0.3 should work for different problems. Determine the optimum number of trees for this learning rate.

In [20]:
%%time

num_round_choice = [10, 20, 40, 100]
eta_choice = [0.05, 0.10, 0.15, 0.20, 0.30]

space = {
    # Optimize
    'num_round': hp.choice('num_round', num_round_choice),
    'eta': hp.choice('eta', eta_choice),
    
    # Fixed    
    'max_depth': baseline_params['max_depth'],
    'min_child_weight': baseline_params['min_child_weight'],
    'subsample': baseline_params['subsample'],
    'gamma': baseline_params['gamma'],
    'colsample_bytree': baseline_params['colsample_bytree'],
    
    **static_params
}


trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=20,
            trials=trials)

SPACE:                                                
{'eval_metric': 'logloss', 'labelCol': 'label', 'eta': 0.05, 'colsample_bytree': 0.9, 'silent': 0, 'gamma': 0.9, 'max_depth': 6, 'subsample': 0.9, 'min_child_weight': 50.0, 'nworkers': 1, 'num_round': 10, 'featuresCol': 'features', 'predictionCol': 'prediction', 'objective': 'binary:logistic', 'nthread': 1}
LOG-LOSS: 0.5883944275615641, ROC-AUC: 0.7181447424350922
SPACE:                                                                          
{'eval_metric': 'logloss', 'labelCol': 'label', 'eta': 0.15, 'colsample_bytree': 0.9, 'silent': 0, 'gamma': 0.9, 'max_depth': 6, 'subsample': 0.9, 'min_child_weight': 50.0, 'nworkers': 1, 'num_round': 40, 'featuresCol': 'features', 'predictionCol': 'prediction', 'objective': 'binary:logistic', 'nthread': 1}
LOG-LOSS: 0.5064321632924346, ROC-AUC: 0.7315063196507958                       
SPACE:                                                                          
{'eval_metric': 'logloss',

LOG-LOSS: 0.500978735342114, ROC-AUC: 0.7389325937145648                          
SPACE:                                                                            
{'eval_metric': 'logloss', 'labelCol': 'label', 'eta': 0.2, 'colsample_bytree': 0.9, 'silent': 0, 'gamma': 0.9, 'max_depth': 6, 'subsample': 0.9, 'min_child_weight': 50.0, 'nworkers': 1, 'num_round': 20, 'featuresCol': 'features', 'predictionCol': 'prediction', 'objective': 'binary:logistic', 'nthread': 1}
LOG-LOSS: 0.508414049995408, ROC-AUC: 0.7284819689710851                         
SPACE:                                                                           
{'eval_metric': 'logloss', 'labelCol': 'label', 'eta': 0.2, 'colsample_bytree': 0.9, 'silent': 0, 'gamma': 0.9, 'max_depth': 6, 'subsample': 0.9, 'min_child_weight': 50.0, 'nworkers': 1, 'num_round': 100, 'featuresCol': 'features', 'predictionCol': 'prediction', 'objective': 'binary:logistic', 'nthread': 1}
LOG-LOSS: 0.501991921677937, ROC-AUC: 0.7375948663159

In [21]:
best

{'eta': 4, 'num_round': 3}

Обратите внимание на то, что в случае с `hp.choice` в переменной `best` хранится не конкретное значение гиперпараметра, а его индекс из списка, например, `num_round_choice`

In [22]:
eta = eta_choice[best['eta']]  # change me!
num_round = num_round_choice[best['num_round']]  # change me!

In [23]:
eta, num_round

(0.3, 100)

In [24]:
# eta, num_round = (0.5, 100)

### Tune tree-specific parameters

> Tune tree-specific parameters ( max_depth, min_child_weight, gamma, subsample, colsample_bytree) for decided learning rate and number of trees. Note that we can choose different parameters to define a tree and I’ll take up an example here.

#### Tune max_depth, min_child_weight

#### Tune gamma

#### Tune subsample, colsample_bytree

In [25]:
static_params = {
    'featuresCol': "features", 
    'labelCol': "label", 
    'predictionCol': "prediction",
    'eval_metric': 'logloss',
    'objective': 'binary:logistic',
    'nthread': 1,
    'silent': 0,
    'nworkers': 4 # changed
}

In [26]:
max_depth_choice = [3, 5, 8]
min_child_weight_choice = [10., 30., 50.]
subsample_choice = [0.4, 0.6, 0.8, 1.0]
gamma_choice = [0.01, 0.3, 0.6, 0.9, 1.5]
colsample_bytree_choice = [0.4, 0.6, 0.8, 1.0]

space = {
    # Fixed
    'num_round': num_round,
    'eta': eta,
    
    # Optimize    
    'max_depth': hp.choice('max_depth', max_depth_choice),
#     'max_depth': baseline_params['max_depth'],
    'min_child_weight': hp.choice('min_child_weight', min_child_weight_choice),
#     'min_child_weight': baseline_params['min_child_weight'],
    'subsample': hp.choice('subsample', subsample_choice),
    'gamma': hp.choice('gamma', gamma_choice),
    'colsample_bytree': hp.choice('colsample_bytree', colsample_bytree_choice),
    
    **static_params
}



In [27]:
trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=30,
            trials=trials)

SPACE:                                                
{'eval_metric': 'logloss', 'labelCol': 'label', 'eta': 0.3, 'colsample_bytree': 0.6, 'silent': 0, 'gamma': 0.6, 'max_depth': 8, 'subsample': 0.8, 'min_child_weight': 50.0, 'nworkers': 4, 'num_round': 100, 'featuresCol': 'features', 'predictionCol': 'prediction', 'objective': 'binary:logistic', 'nthread': 1}
LOG-LOSS: 0.5001380929596146, ROC-AUC: 0.7400881270560927
SPACE:                                                                          
{'eval_metric': 'logloss', 'labelCol': 'label', 'eta': 0.3, 'colsample_bytree': 0.6, 'silent': 0, 'gamma': 0.01, 'max_depth': 3, 'subsample': 0.4, 'min_child_weight': 10.0, 'nworkers': 4, 'num_round': 100, 'featuresCol': 'features', 'predictionCol': 'prediction', 'objective': 'binary:logistic', 'nthread': 1}
LOG-LOSS: 0.5053556260433809, ROC-AUC: 0.7329379070481008                       
SPACE:                                                                          
{'eval_metric': 'logloss'

LOG-LOSS: 0.5003154688571911, ROC-AUC: 0.7396303378279911                        
SPACE:                                                                           
{'eval_metric': 'logloss', 'labelCol': 'label', 'eta': 0.3, 'colsample_bytree': 1.0, 'silent': 0, 'gamma': 0.3, 'max_depth': 3, 'subsample': 1.0, 'min_child_weight': 50.0, 'nworkers': 4, 'num_round': 100, 'featuresCol': 'features', 'predictionCol': 'prediction', 'objective': 'binary:logistic', 'nthread': 1}
LOG-LOSS: 0.5051391469287327, ROC-AUC: 0.7332046900292287                        
SPACE:                                                                           
{'eval_metric': 'logloss', 'labelCol': 'label', 'eta': 0.3, 'colsample_bytree': 0.6, 'silent': 0, 'gamma': 0.6, 'max_depth': 3, 'subsample': 0.4, 'min_child_weight': 10.0, 'nworkers': 4, 'num_round': 100, 'featuresCol': 'features', 'predictionCol': 'prediction', 'objective': 'binary:logistic', 'nthread': 1}
LOG-LOSS: 0.5051051454064062, ROC-AUC: 0.7333901150635

In [28]:
best

{'colsample_bytree': 2,
 'gamma': 0,
 'max_depth': 2,
 'min_child_weight': 1,
 'subsample': 2}

In [29]:
colsample_bytree = colsample_bytree_choice[best['colsample_bytree']]
gamma = gamma_choice[best['gamma']]
max_depth = max_depth_choice[best['max_depth']]
min_child_weight = min_child_weight_choice[best['min_child_weight']]
subsample = subsample_choice[best['subsample']]

In [30]:
colsample_bytree, gamma, max_depth, min_child_weight, subsample

(0.8, 0.01, 8, 30.0, 0.8)

### Tune regularization parameters

> Tune regularization parameters (lambda, alpha) for xgboost which can help reduce model complexity and enhance performance.

In [31]:
lambda_choice = [0., 0.01, 0.1, 1., 10.]
alpha_choice = [0., 0.01, 0.1, 1., 10.]

space = {
    # Optimize
    'alpha': hp.choice('alpha', alpha_choice),
    'reg_lambda': hp.choice('lambda', lambda_choice),
    
    # Fixed
    'num_round': num_round,
    'eta': eta, 
    'max_depth': max_depth,
    'min_child_weight': min_child_weight,
    'subsample': subsample,
    'gamma': gamma,
    'colsample_bytree': colsample_bytree,
    
    **static_params
}


In [32]:
trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=20,
            trials=trials)

SPACE:                                                
{'eval_metric': 'logloss', 'labelCol': 'label', 'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0.01, 'max_depth': 8, 'eta': 0.3, 'lambda': 0.0, 'min_child_weight': 30.0, 'nworkers': 4, 'num_round': 100, 'featuresCol': 'features', 'predictionCol': 'prediction', 'objective': 'binary:logistic', 'nthread': 1, 'silent': 0, 'alpha': 10.0}
LOG-LOSS: 0.49977271378089266, ROC-AUC: 0.7404214183140699
SPACE:                                                                           
{'eval_metric': 'logloss', 'labelCol': 'label', 'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0.01, 'max_depth': 8, 'eta': 0.3, 'lambda': 0.1, 'min_child_weight': 30.0, 'nworkers': 4, 'num_round': 100, 'featuresCol': 'features', 'predictionCol': 'prediction', 'objective': 'binary:logistic', 'nthread': 1, 'silent': 0, 'alpha': 0.0}
LOG-LOSS: 0.49985367917423307, ROC-AUC: 0.7403588273786044                       
SPACE:                                     

LOG-LOSS: 0.49985672607224496, ROC-AUC: 0.7402958756323418                       
SPACE:                                                                           
{'eval_metric': 'logloss', 'labelCol': 'label', 'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0.01, 'max_depth': 8, 'eta': 0.3, 'lambda': 1.0, 'min_child_weight': 30.0, 'nworkers': 4, 'num_round': 100, 'featuresCol': 'features', 'predictionCol': 'prediction', 'objective': 'binary:logistic', 'nthread': 1, 'silent': 0, 'alpha': 0.0}
LOG-LOSS: 0.49984020430460807, ROC-AUC: 0.740088447220825                        
SPACE:                                                                           
{'eval_metric': 'logloss', 'labelCol': 'label', 'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0.01, 'max_depth': 8, 'eta': 0.3, 'lambda': 10.0, 'min_child_weight': 30.0, 'nworkers': 4, 'num_round': 100, 'featuresCol': 'features', 'predictionCol': 'prediction', 'objective': 'binary:logistic', 'nthread': 1, 'silent': 0, 'alpha'

In [33]:
best

{'alpha': 2, 'lambda': 1}

In [34]:
alpha = alpha_choice[best['alpha']]
reg_lambda = lambda_choice[best['lambda']]

In [35]:
alpha, reg_lambda

(0.1, 0.01)

### Lower the learning rate and decide the optimal parameters

In [36]:
params = {
    # Optimize
    'alpha': alpha,
    'reg_lambda': reg_lambda,
    'num_round': num_round,
    'eta': eta, 
    'max_depth': max_depth,
    'min_child_weight': min_child_weight,
    'subsample': subsample,
    'gamma': gamma,
    'colsample_bytree': colsample_bytree,
    
    **static_params
}

In [37]:
params

{'alpha': 0.1,
 'colsample_bytree': 0.8,
 'eta': 0.3,
 'eval_metric': 'logloss',
 'featuresCol': 'features',
 'gamma': 0.01,
 'labelCol': 'label',
 'max_depth': 8,
 'min_child_weight': 30.0,
 'nthread': 1,
 'num_round': 100,
 'nworkers': 4,
 'objective': 'binary:logistic',
 'predictionCol': 'prediction',
 'reg_lambda': 0.01,
 'silent': 0,
 'subsample': 0.8}

In [38]:
best_params_model = XGBoostEstimator(**params).fit(train_df)

In [39]:
best_params_test_metrics = {
    'logloss': logloss(best_params_model, test_df, probabilities_col='probabilities'),
    'rocauc': rocauc(best_params_model, test_df, probabilities_col='probabilities')
}

all_metrics['xgboost_best_params'] = best_params_test_metrics

In [40]:
get_ate(all_metrics, CONTROL_NAME)

,metric,xgboost_best_params ate %
0,rocauc,1.872134
1,logloss,-2.082839


---
## LogisticRegression Tuning

Подберем гиперпараметры для логрега из предыдущих практик

In [41]:
def objective(space, estimatorModel = LogisticRegression):
    estimator = estimatorModel(**space)
    success = False
    attempts = 0
    model = None
    while not success and attempts < 2:
        try:
            model = estimator.fit(train_df)
            success = True
        except Exception as e:
            attempts += 1
            print(e)
            print('Try again')
        
    log_loss = logloss(model, val_df, probabilities_col='probability')
    roc_auc = rocauc(model, val_df, probabilities_col='probability')
    
    print('LOG-LOSS: {}, ROC-AUC: {}'.format(log_loss, roc_auc))

    return {'loss': log_loss, 'rocauc': roc_auc, 'status': STATUS_OK }

In [42]:
lr_baseline_params = {
    'maxIter': 10,
    'regParam': 0.0,
    'elasticNetParam': 0.0,
    'tol': 1e-06
}

lr_baseline_model = LogisticRegression(**lr_baseline_params).fit(train_df)

In [43]:
baseline_test_metrics = {
    'logloss': logloss(lr_baseline_model, test_df, probabilities_col='probability'),
    'rocauc': rocauc(lr_baseline_model, test_df, probabilities_col='probability')
}

all_metrics['lr baseline'] = baseline_test_metrics

In [44]:
max_iter_choice = [10, 20, 30]
reg_param_choice = [0.0, 0.01, 0.1, 0.5, 1.0]
elastic_net_param_choice = [0.0, 0.01, 0.1, 0.5, 1.0]

space = {
    'maxIter': hp.choice('max_iter', max_iter_choice),
    'regParam': hp.choice('reg_param', reg_param_choice),
    'elasticNetParam': hp.choice('elastic_net_param', elastic_net_param_choice)
}

In [45]:
trials = Trials()
best = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=36,
    trials=trials
)

LOG-LOSS: 0.5752172500206219, ROC-AUC: 0.5            
LOG-LOSS: 0.5696866960748407, ROC-AUC: 0.6651109716451596                       
LOG-LOSS: 0.5294676130310666, ROC-AUC: 0.7030947639809677                       
LOG-LOSS: 0.549456182583188, ROC-AUC: 0.7008628035451681                        
LOG-LOSS: 0.549456182583188, ROC-AUC: 0.7008628035451712                        
LOG-LOSS: 0.5278837426902139, ROC-AUC: 0.7021667489839274                       
LOG-LOSS: 0.5752172500206219, ROC-AUC: 0.5                                      
LOG-LOSS: 0.5640565126220034, ROC-AUC: 0.6648189706296547                       
LOG-LOSS: 0.5283011885933424, ROC-AUC: 0.7032901003424322                       
LOG-LOSS: 0.5354273561551044, ROC-AUC: 0.7019957367513623                       
LOG-LOSS: 0.5752172500206219, ROC-AUC: 0.5                                       
LOG-LOSS: 0.5423469736856984, ROC-AUC: 0.6882960416345602                        
LOG-LOSS: 0.5640780128301158, ROC-AUC: 0.68996380775

In [46]:
best

{'elastic_net_param': 1, 'max_iter': 1, 'reg_param': 1}

In [47]:
max_iter = max_iter_choice[best['max_iter']]
reg_param = reg_param_choice[best['reg_param']]
elastic_net_param = elastic_net_param_choice[best['elastic_net_param']]

lr_params = {
    'maxIter': max_iter,
    'regParam': reg_param,
    'elasticNetParam': elastic_net_param
}
lr_params

{'elasticNetParam': 0.01, 'maxIter': 20, 'regParam': 0.01}

In [48]:
lr_best_params_model = LogisticRegression(**lr_params).fit(train_df)

In [49]:
lr_best_params_test_metrics = {
    'logloss': logloss(lr_best_params_model, test_df, probabilities_col='probability'),
    'rocauc': rocauc(lr_best_params_model, test_df, probabilities_col='probability')
}

all_metrics['lr_best_params'] = lr_best_params_test_metrics

In [50]:
get_ate(all_metrics, CONTROL_NAME)

,lr baseline ate %,lr_best_params ate %,metric,xgboost_best_params ate %
0,-3.274936,-3.392148,rocauc,1.872134
1,3.372509,3.306220,logloss,-2.082839


---
## Optional [MongoTrials](https://hyperopt.github.io/hyperopt/scaleout/mongodb/)

> For parallel search, hyperopt includes a MongoTrials implementation that supports asynchronous updates.

**TLDR** Преимущества использования `MongoTrials`:
* `MongoTrials` позволяет параллельно запускать несколько вычислений целевой функции
* Динамический уровень параллелизма - можно добавлять/удалять воркеров, которые вычисляют целевую функцию
* Все результаты сохраняются в БД - история запусков никуда не потеряется

*За выполнение данного задания можно получить дополнительно +0.4 к итоговому баллу*

### XGBoost Tuning

In [ ]:
######################################
######### YOUR CODE HERE #############
######################################

# Results

Подведем итоги.

Обучите модели с найденными (оптимальными) гиперпараметрами и сделайте справнение на отложенной выборке

In [51]:
all_metrics = {}

In [52]:
baseline_model = XGBoostEstimator(**{**static_params, **baseline_params}).fit(train_df)

In [53]:
baseline_test_metrics = {
    'logloss': logloss(baseline_model, test_df, probabilities_col='probabilities'),
    'rocauc': rocauc(baseline_model, test_df, probabilities_col='probabilities')
}

all_metrics[CONTROL_NAME] = baseline_test_metrics

In [54]:
best_params_model = XGBoostEstimator(**params).fit(train_df)

In [55]:
best_params_test_metrics = {
    'logloss': logloss(best_params_model, test_df, probabilities_col='probabilities'),
    'rocauc': rocauc(best_params_model, test_df, probabilities_col='probabilities')
}

all_metrics['xgboost_best_params'] = best_params_test_metrics

In [56]:
lr_best_params_model = LogisticRegression(**lr_params).fit(train_df)

In [57]:
lr_best_params_test_metrics = {
    'logloss': logloss(lr_best_params_model, test_df, probabilities_col='probability'),
    'rocauc': rocauc(lr_best_params_model, test_df, probabilities_col='probability')
}

all_metrics['lr_best_params'] = lr_best_params_test_metrics

Итоговая таблица

In [58]:
get_ate(all_metrics, CONTROL_NAME)

,lr_best_params ate %,metric,xgboost_best_params ate %
0,-3.322021,rocauc,1.972038
1,3.256206,logloss,-2.152145
